In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
sys.path.insert(0, "..")

In [2]:
import trimesh
import matplotlib
import numpy as np
import meshplot as mp
import matplotlib.pyplot as plt

# Visualize the Implicit Field

In [3]:
!ls ../pretrained/neural_fields/

armadillo  bunny   cylinder  half_noisy_sphere	noisy_torus
bar	   cactus  dino      jolteon


In [4]:
log_path = "../pretrained/neural_fields/jolteon"

In [5]:
import yaml
import argparse
import os.path as osp
from train import dict2namespace
from pprint import pprint
from utils import load_imf

original, cfg = load_imf(
    log_path, 
    config_fpath=osp.join(log_path, "config.yaml"), 
    return_cfg=True)
original

../utils.py:49: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = dict2namespace(yaml.load(f))


../pretrained/neural_fields/jolteon/latest.pt
Net:
Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
  (act): Sine()
)


Net(
  (blocks): ModuleList(
    (0): Linear(in_features=3, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Linear(in_features=512, out_features=1, bias=True)
  )
  (act): Sine()
)

In [6]:
import numpy as np
import torch
from tqdm import tqdm
import skimage
import skimage.measure
from trainers.utils.vis_utils import imf2mesh
    
original.eval()
res = 256
new_mesh = imf2mesh(original, res=res, threshold=0.00, normalize=True, norm_type='res')
new_mesh

100%|██████████| 1678/1678 [00:06<00:00, 257.42it/s]


<trimesh.Trimesh(vertices.shape=(89767, 3), faces.shape=(179526, 3))>

In [7]:
# verts = (new_mesh.vertices * 2 - res) / float(res)
# new_mesh = trimesh.Trimesh(vertices=verts, faces=new_mesh.faces)
print(new_mesh.vertices.max(), new_mesh.vertices.min())
mp.plot(new_mesh.vertices, new_mesh.faces)

0.7369110584259033 -0.683293342590332


/home/grendelyang/anaconda3/envs/IGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/grendelyang/anaconda3/envs/IGP/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "uint32" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003545…

# Creating Handles

1. Creating a set of deforming handles (i.e. sparse points on the surface that will change according to the shape)
2. Creating a set of statisc handles (i.e. sparse points on the surface that will remain unchanged)

In [10]:
verts, faces = new_mesh.vertices, new_mesh.faces
print(verts.max(axis=0))
print(verts.min(axis=0))

[0.42319047 0.60385621 0.73691106]
[-0.4224813  -0.68329334 -0.5138064 ]


In [30]:
from scipy.spatial.transform import Rotation as ROT

# neg_vert = np.concatenate([
#     verts[
#         np.logical_and(verts[:, 1] < -0.65, verts[:, 2] > 0), :],
#     verts[
#         np.logical_and(verts[:, 1] < -0.65, np.logical_and(verts[:, 2] < 0, verts[:, 0] < 0)), :]
# ], axis=0)
# pos_vert = verts[
#     np.logical_and(verts[:, 1] < -0.65, np.logical_and(verts[:, 2] < 0, verts[:, 0] > 0)), 
#     :]

neg_vert = verts[np.logical_and(verts[:, 1] < -0.65, verts[:, 2] > 0), :]
pos_vert = verts[np.logical_and(verts[:, 1] < -0.65, verts[:, 2] < 0), :]

# Rotate around the a center, with certain eular angle
rot_degree = -45
rot_axis = 'x'
rot_mat = ROT.from_euler(rot_axis, rot_degree, degrees=True).as_matrix()
# center = np.array([0, -0.35, -0.05]).reshape(1, 3)
center = np.array([0, -0., -0.0]).reshape(1, 3)
translate = np.array([0., 0.0, -0.1]).reshape(1, 3)
pos_vert_transf = np.matmul(pos_vert - center, rot_mat) + center + translate

p = mp.plot(new_mesh.vertices, new_mesh.faces)
p.add_points(neg_vert, shading={"point_color": "blue", "point_size": 0.05})
p.add_points(pos_vert, shading={"point_color": "red", "point_size": 0.05})
p.add_points(center, shading={"point_color": "black", "point_size": 0.3})
p.add_points(pos_vert_transf, shading={"point_color": "green", "point_size": 0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003545…

4

# Compare to Open3D ASAP Algorithm

In [31]:
from trainers.utils.o3d_deformation import deform_mesh_o3d, trimesh_to_o3dmesh, o3dmesh_to_trimesh

handles = np.concatenate([neg_vert, pos_vert], axis=0)
targets = np.concatenate([neg_vert, pos_vert_transf], axis=0)

mesh_prime_tri = deform_mesh_o3d(original, handles, targets, imf_mesh = new_mesh, 
                                 steps=1000, smoothed_alpha=1.)


[Open3D DEBUG] [DeformAsRigidAsPossible] setting up S'
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up S'
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up system matrix L
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up system matrix L
[Open3D DEBUG] [DeformAsRigidAsPossible] setting up sparse solver
[Open3D DEBUG] [DeformAsRigidAsPossible] done setting up sparse solver
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=0, energy=5.911776e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=1, energy=5.911731e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=2, energy=5.911672e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=3, energy=5.911620e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=4, energy=5.911566e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=5, energy=5.911514e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=6, energy=5.911462e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=7, energy=5.911410e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=8, energy

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=113, energy=5.907073e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=114, energy=5.907036e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=115, energy=5.907000e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=116, energy=5.906964e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=117, energy=5.906928e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=118, energy=5.906891e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=119, energy=5.906855e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=120, energy=5.906819e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=121, energy=5.906783e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=122, energy=5.906747e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=123, energy=5.906711e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=124, energy=5.906675e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=125, energy=5.906639e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=126, energy=5.906603e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=229, energy=5.903077e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=230, energy=5.903044e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=231, energy=5.903012e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=232, energy=5.902979e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=233, energy=5.902946e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=234, energy=5.902913e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=235, energy=5.902880e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=236, energy=5.902848e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=237, energy=5.902815e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=238, energy=5.902782e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=239, energy=5.902750e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=240, energy=5.902717e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=241, energy=5.902685e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=242, energy=5.902652e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=347, energy=5.899347e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=348, energy=5.899317e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=349, energy=5.899287e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=350, energy=5.899256e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=351, energy=5.899226e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=352, energy=5.899195e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=353, energy=5.899165e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=354, energy=5.899135e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=355, energy=5.899104e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=356, energy=5.899074e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=357, energy=5.899044e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=358, energy=5.899013e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=359, energy=5.898983e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=360, energy=5.898953e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=463, energy=5.895923e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=464, energy=5.895895e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=465, energy=5.895866e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=466, energy=5.895838e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=467, energy=5.895809e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=468, energy=5.895781e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=469, energy=5.895752e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=470, energy=5.895724e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=471, energy=5.895695e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=472, energy=5.895667e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=473, energy=5.895638e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=474, energy=5.895610e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=475, energy=5.895581e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=476, energy=5.895553e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=580, energy=5.892681e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=581, energy=5.892654e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=582, energy=5.892628e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=583, energy=5.892601e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=584, energy=5.892574e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=585, energy=5.892547e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=586, energy=5.892520e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=587, energy=5.892494e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=588, energy=5.892467e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=589, energy=5.892440e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=590, energy=5.892414e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=591, energy=5.892387e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=592, energy=5.892360e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=593, energy=5.892334e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=699, energy=5.889593e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=700, energy=5.889568e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=701, energy=5.889542e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=702, energy=5.889517e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=703, energy=5.889492e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=704, energy=5.889467e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=705, energy=5.889442e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=706, energy=5.889417e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=707, energy=5.889393e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=708, energy=5.889368e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=709, energy=5.889343e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=710, energy=5.889318e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=711, energy=5.889293e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=712, energy=5.889268e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=816, energy=5.886761e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=817, energy=5.886738e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=818, energy=5.886715e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=819, energy=5.886692e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=820, energy=5.886668e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=821, energy=5.886645e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=822, energy=5.886622e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=823, energy=5.886599e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=824, energy=5.886575e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=825, energy=5.886552e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=826, energy=5.886529e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=827, energy=5.886506e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=828, energy=5.886483e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=829, energy=5.886460e-01
[Open3

[Open3D DEBUG] [DeformAsRigidAsPossible] iter=933, energy=5.884136e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=934, energy=5.884114e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=935, energy=5.884093e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=936, energy=5.884071e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=937, energy=5.884050e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=938, energy=5.884028e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=939, energy=5.884007e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=940, energy=5.883986e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=941, energy=5.883964e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=942, energy=5.883943e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=943, energy=5.883921e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=944, energy=5.883900e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=945, energy=5.883878e-01
[Open3D DEBUG] [DeformAsRigidAsPossible] iter=946, energy=5.883857e-01
[Open3

In [32]:
p = mp.plot(mesh_prime_tri.vertices, mesh_prime_tri.faces)
p.add_points(neg_vert, shading={"point_color": "blue", "point_size": 0.05})
p.add_points(pos_vert, shading={"point_color": "red", "point_size": 0.05})
p.add_points(pos_vert_transf, shading={"point_color": "green", "point_size": 0.05})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0003544…

3

In [33]:
mesh_prime_o3d = trimesh_to_o3dmesh(mesh_prime_tri)
# o3d_pcl = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(targets))
# mesh_prime_o3d.compute_vertex_normals()
# mesh_prime_o3d.paint_uniform_color([0.1, 0.5, 0.1])
# o3d_mesh0 = trimesh_to_o3dmesh(new_mesh)
# o3d_mesh0.paint_uniform_color([0.1, 0.1, 0.5])
# o3d.visualization.draw_geometries([mesh_prime_o3d, o3d_mesh0, o3d_pcl])

In [34]:
# Saving it to data
out_dir = "../data/jolteon/deform_%s_rotate%d.npy" % (rot_axis, rot_degree)
np.save(out_dir, {
    "handles": handles,
    "targets": targets,
    'gtr_verts': mesh_prime_tri.vertices,
    'gtr_faces': mesh_prime_tri.faces
})
out_dir

FileNotFoundError: [Errno 2] No such file or directory: '../data/arap_dino/deform_x_rotate-45.npy'